In [8]:
import pandas as pd
import numpy as np
import os

##### match two dataSet #######
def datasetMatch(MOFName):
    dfML= dfMLOrigin[dfMLOrigin['MOF'].isin(MOFName)]
    matchedMOFIndex=np.isin(MOFName, dfML['MOF'].values)
    return matchedMOFIndex, dfML


###### read ML data ######
dfMLOrigin = pd.read_excel('data/ML_data/descriptor_4717MOF.xlsx')

###### read flexibility data ##### 
flexibilityList=os.listdir('data/flexibility_data/y_data/adsorption_data')
flexivilityData=[]

for i, name in enumerate(flexibilityList):   
    df=pd.read_csv('data/flexibility_data/y_data/adsorption_data/'+name)
    rigidValue=np.array(df[df.columns[1]],dtype=float)
    flexValue=np.mean(np.array(df[df.columns[2:]],dtype=float),axis=1)
    label=np.array([name.split("_")[1] for x in range(0,len(flexValue))],dtype=str)
    singleSet=np.column_stack([rigidValue,flexValue,label])

    if i==0:
        MOFName=np.array(df[df.columns[0]],dtype='U6')
        matchedMOFIndex, dfML=datasetMatch(MOFName)
        flexibilityData=singleSet[matchedMOFIndex,:].copy()
    else:
        flexibilityData=np.concatenate([flexibilityData.copy(),singleSet[matchedMOFIndex,:].copy()])
 

print(dfML.head(2))
print(flexibilityData[0:3,:])

# dfML recored 88 MOF's 1024 features (before dimension reduction)
# flexibilityData recored [88*9,3] y data 
# (1st column as rigid, 2nd column as flexible_mean, 3rd column as adsorbate label)

    ID     MOF Periodic Chemical Formula  ρ(g.cm-3)  PLD (Å)  LCD (Å)  \
12  13  ABUWOJ                H7C12O7Zn2    1.15833  4.03039  5.07969   
24  25  ACOLIP              H19C22N5O4Zn    1.04949  3.57647  4.91034   

    VSA  (m2/cm3)  GSA (m2/g)        vf  vp (cm3/g)  ...  F01[Ne-Rh]  \
12        1007.55     869.832  0.545974    0.532253  ...           0   
24           0.00       0.000  0.454051    0.521040  ...           0   

    F01[Ne-Pd]  F01[Ne-Ag]  F01[Ne-Cd]  F01[Ne-In]  F01[Ne-Sn]  F01[Ne-Sb]  \
12           0           0           0           0           0           0   
24           0           0           0           0           0           0   

    F01[Ne-Te]  F01[Ne-I]  F01[Ne-Xe]  
12           0          0           0  
24           0          0           0  

[2 rows x 1024 columns]
[['5.395625473' '5.016580520775' 'xenon']
 ['5.788896266' '4.817801024044944' 'xenon']
 ['2.323461698' '3.3505466442417773' 'xenon']]


In [11]:
### manually add three adsorbatedescriptor####

adsorbateData=[['xenon',], ['butane'], ['propene'], ['ethane'], ['propane'], ['CO2'], ['ethene'], ['methane'],
 ['krypton']]